# Deep learning tutorial: particle identification

Welcome to this tutorial on deep learning for particle identification in neutrino events! In this tutorial, you will learn how to define and run deep-learning methods to identify particles in neutrino events.

Deep learning is a powerful subset of machine learning that involves training complex artificial neural networks to solve problems that traditional machine-learning techniques may struggle with. In this tutorial, we will dive deeper into the world of deep learning and explore how to train a neural network to identify particles in neutrino events. By the end of this tutorial, you will have a solid understanding of deep learning concepts and how they can be applied in a practical setting. So let's get started!

##Prerequisites

To start off, let's check if a GPU is available and turn it on if it is. Using a GPU can significantly speed up the training process of our neural network, so it's always a good idea to take advantage of it if possible:

```
Edit -> Notebook settings -> Hardware accelerator: GPU -> Save.
```



Before we can begin training our neural network, we need to download the dataset and import the necessary Python packages and modules.

Downloading the dataset will provide us with the data we need to train and test our neural network. Additionally, we'll need to import various packages and modules such as NumPy, Pandas, Matplotlib, and TensorFlow to handle the data, visualize it, and define and train our neural network.

Let's get started by downloading the dataset and importing the required packages and modules:

In [ ]:
!wget "https://cernbox.cern.ch/remote.php/dav/public-files/BDuycrUjVffTCxJ/modules.py"
!wget "https://cernbox.cern.ch/remote.php/dav/public-files/IAxTMwkrAhFKFGz/df_tutorial.p"

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from modules import *

Before we proceed, let's check if the GPU was successfully detected. Using a GPU can significantly accelerate the training process of our neural network, so it's important to ensure that it's working properly.

Let's run a quick check to confirm if the GPU is available and ready to use:

In [ ]:
# Check if a GPU is available and turn it on if possible
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    # Configure the GPU device
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU device not found')
    print('Found GPU at: {}'.format(device_name))
else:
    print("No GPU detected. Using CPU instead.")

##Dataset

Now that we have downloaded the dataset, we can proceed to load it into our notebook. This dataset contains the necessary information for training and testing our neural network.

Let's load the dataset into our notebook so that we can start preparing it for use in our deep-learning model:

In [ ]:
# Read dataframe
df = pd.read_pickle('df_tutorial.p')

Let's take a look at the dataset we'll be working with. The dataset consists of 59,578 particle gun events from the [SFGD detector of the T2K experiment](https://doi.org/10.1088/1748-0221/15/12/p12003), each with the following attributes:

- **TruePID**: PDG code for particle identification (PID); 2212 (proton), 13 (muon), 211 (pion).
- **TrueMomentum**: momentum in MeV/c.
- **NNodes**: number of nodes of the event (3D spatial points).
- **NodeOrder**: order of the nodes within the event.
- **NodePosX**: array with the coordinates of the nodes along the X-axis (in mm).
- **NodePosY**: array with the coordinates of the nodes along the Y-axis (in mm).
- **NodePosZ**: array with the coordinates of the nodes along the Z-axis (in mm).
- **NodeT**: array with the timestamps of the nodes (in ns).
- **Nodededx**: array with energy deposits of the nodes (dE/dx).
- **TrkLen**: length of the track (in mm).
- **TrkEDepo**: total track energy deposition (in arbitrary units).
- **TrkDir1**: track direction, polar angle (theta, in radians).
- **TrkDir2**: track direction, azimuth angle (phi, in radians).

These attributes will be used to train a neural network to identify particles in neutrino events. Let's take a first look at the dataset by running the following cell:

In [ ]:
# Print the first few rows of the dataset
df.head()

We can also check the correlations among the variables, with the exception of the node features, since each event has a different number of nodes:

In [ ]:
# Print variable correlations
df.corr()

The 3D spatial points of the events are typically represented as nodes, which correspond to fitted positions after track reconstruction. To visualize the events, we can plot the nodes in the detector space. By default, the code displays the nodes for the first event (event 0), but we can visualize nodes for other events by adjusting the event_number variable.

In [ ]:
# Set the event number to plot
event_number = 0

# Call the plot_event function to display the nodes in the detector space for the chosen event
plot_event(df, event_number)

Preprocessing the data is essential regardless of the data type and algorithm chosen. The goal is to prepare the data to make it understandable for the machine learning algorithm. In our case, we want to learn to predict a label **y** from a fixed-size vector of features **X**. However, the input data is in 3D, and every event (track) has a different size, making it unsuitable for most machine learning algorithms.

To fix this, we need to extract a fixed number of features from each event. One approach is to use `TrkLen` and `TrkEDepo` as features. This approach reduces the dimensionality of the data and makes it more suitable for machine learning algorithms.

In addition, we will encode the particle identification (PID) codes for protons (2212), muons (13), and pions (211) into 0, 1, and 2, respectively.

In [ ]:
# Create input X and labels y from the dataset
X = np.zeros(shape=(len(df), 2), dtype=np.float32)
y = np.zeros(shape=(len(df),), dtype=np.int32)

# Fill dataset
for event_n, event in df.iterrows():
    
    # Encode the PID label
    pid_label = event['TruePID']
    if pid_label == 2212: 
        pid_label = 0  # proton
    elif pid_label == 13: 
        pid_label = 1  # muon
    else:
        pid_label = 2  # pion
    
    # Retrieve the first node and assign it to X
    X[event_n, 0] = event['TrkLen']
    X[event_n, 1] = event['TrkEDepo']

    # Assign the encoded PID label to y
    y[event_n] = pid_label

# Standardize the dataset (mean=0, std=1)
X_stan = scale(X)

Visualizing the training data is an important step in understanding the data before training the model. To simplify our analysis, we can start by comparing only protons and muons (ignoring pions for the moment). One way to achieve this is to create a scatter plot of one feature against the other:

In [ ]:
# Define the parameters and labels to plot
param_names = ['TrkLen', 'TrkEDepo']
y_names = ['proton', 'muon']

# Plot the data points for protons and muons
plot_params_pid(X[y!=2], y[y!=2], param_names, y_names)

Good! The scatter plot shows that protons and muons have distinct distributions and can be visually separated from each other.

## Fully connected neural networks

Training a deep learning algorithm is a challenging task that requires a careful approach. Typically, the algorithm learns from a training dataset until it can make accurate predictions on new, unseen data. To test how well the algorithm generalizes to new data, we split the original dataset into two groups (although sometimes it's divided into three groups, for simplicity, we will be using a two-group approach in this example):

- Training set: The model learns from this set only, and it should be the largest set.
- Test set: We use this set to evaluate the model once it's fully trained.

In this example, we'll allocate 60% of the data to the training set and 40% to the test set. To avoid introducing any bias during training, we'll shuffle the training examples before feeding them to the algorithm.

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_stan[y!=2], y[y!=2], test_size=0.4, 
                                                    random_state=7, shuffle=True)

# Print the number of examples in each set
print("Number of training examples: ", len(X_train))
print("Number of test examples: ", len(X_test))

This tutorial aims to explore deep learning methods, which is a subset of machine learning that involves artificial neural networks. In this tutorial, we will implement a fully connected neural network using the Keras interface from the TensorFlow deep-learning framework. Keras is an ideal API for neural-network prototyping, as it provides high-level abstractions that make it easy to define, train, and evaluate complex models.

In the architecture of our neural network, each neuron computes the function $\sigma(w x + b)$, where $w$ and $b$ are the input weight and bias of the neuron, respectively, and $\sigma$ is the [activation function](https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6). The goal of the activation function is to introduce non-linearity into the model, which allows the neural network to learn more complex relationships between the input data and the output.

Since it's a binary classification task (proton or muon), **the output neuron has a sigmoid activation** function. The [sigmoid activation](https://machinelearningmastery.com/a-gentle-introduction-to-sigmoid-function/) function is a commonly used function in binary classification tasks like the one we have here, where the output must be either 0 or 1. The function maps any input value to a value between 0 and 1, and it has a characteristic S-shaped curve. The output can be interpreted as a probability of belonging to the positive class, where values close to 0 represent high probability of belonging to the negative class and values close to 1 represent high probability of belonging to the positive class.

<div>
<img src="https://cernbox.cern.ch/remote.php/dav/public-files/TCMgCAYrreRyNJv/dense_nn.png?scalingup=0&preview=1&a=1&c=284811006121607168%3A5d14ad7d&x=1920&y=1920" width="700"/>
</div>

For the loss function, we will use the [binary cross-entropy loss](https://machinelearningmastery.com/cross-entropy-for-machine-learning/) function. Binary cross-entropy loss is used for binary classification problems and it measures the difference between the predicted probability distribution and the true probability distribution. The true probability distribution has values of either 0 or 1, depending on the class label, and the predicted probability distribution is given by the output of the sigmoid activation function. The binary cross-entropy loss is defined as:

$$\mathrm{Binary\ Cross-Entropy\ Loss}=\frac{1}{m}\sum_{i=1}^m y_i\log(\hat{y_i}) + (1-y_i)\log(1-\hat{y_i})$$

where $m$ is the number of samples in the dataset, $y_i$ is the true label of the $i$-th sample, $\hat{y_i}$ is the predicted probability of the $i$-th sample belonging to the positive class (i.e., class 1), and $\log$ denotes the natural logarithm. This loss function is commonly used in binary classification problems where we aim to minimize the difference between the predicted and true class labels.

In this code cell, we create and compile the fully connected neural network model using the Keras interface of TensorFlow. We use the Adam optimizer, which is a variant of Stochastic Gradient Descent (SGD), to update the weights during training:

In [ ]:
tf.random.set_seed(7) # for reproducibility

num_features = 2 # TrkLen, TrkEDepo
num_classes = 1 # one output unit is enough since it's a binary classification problem

# Create the fully connected neural network model
input_layer = Input(shape=(num_features,)) # input layer
hidden_layer_1 = Dense(10, activation='relu')(input_layer) # hidden layer 1
hidden_layer_2 = Dense(10, activation='relu')(hidden_layer_1) # hidden layer 2
output_layer = Dense(num_classes, activation='sigmoid')(hidden_layer_2) # output layer
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

We will train the fully connected neural network model with the following hyperparameters: 10 epochs and a batch size of 128. Let's take a moment to define some important terms used in deep learning:

* Batch: a batch is a set of $n$ input examples (also called mini-batch) processed independently and in parallel. During training, a batch results in only one update to the model, which consists of one forward pass (prediction) and one backward pass (adjusting the model weights).

* Epoch: one epoch is one forward and backward pass of all the training examples. In other words, an epoch is one pass over the entire dataset, and it is used to separate training into distinct phases. For a dataset consisting of $m$ training examples and a batch size of $n$, it will take $\lceil \frac{m}{n} \rceil$ iterations to complete one epoch.

In our case, we have 25468 events for training, and we will use a batch size of 128. Therefore, we need 199 iterations to complete one epoch. After 10 epochs, the model will have seen the training data 10 times and made updates to its weights based on the training examples.

Let's train our model:

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1)

It's also common to calculate some metrics to evaluate our deep-learning method's performance on the test set.

In [ ]:
# Predict classes on test set using the trained model
y_pred = model.predict(X_test).round()

# Calculate and print overall accuracy
overall_accuracy = accuracy_score(y_test, y_pred)
print("Overall accuracy: {:2.3}\n".format(overall_accuracy))

# Calculate and print accuracy for proton class
proton_accuracy = accuracy_score(y_test[y_test==0], y_pred[y_test==0])
print(" - Proton accuracy: {:2.3}".format(proton_accuracy))

# Calculate and print accuracy for muon class
muon_accuracy = accuracy_score(y_test[y_test==1], y_pred[y_test==1])
print(" - Muon accuracy: {:2.3}\n".format(muon_accuracy))

# Calculate and plot confusion matrix
conf_matrix = confusion_matrix(y_pred, y_test)
print_conf(conf_matrix, ['protons', 'muons'])

Nice! We're getting almost perfect separation using only two input parameters!

Is there any room for improving the current results? A more robust but straightforward way of making the input data interpretable for the algorithm is to keep the information of only a few nodes of each track. Our preprocessing is illustrated in the following figure (there are many combinations, we are showing just one practical example here):

<div>
<img src="https://cernbox.cern.ch/remote.php/dav/public-files/dmklFFGAgXGLNbL/reg.png?scalingup=0&preview=1&a=1&c=284811005584736256%3A78b96dfb&x=1920&y=1920" width="500"/>
</div>

where we keep the dE/dx of the first 3 and last 5 nodes of each track, along with their 4 global parameters, building up an array of size 12. In cases where the track has less than 8 nodes, we fill the empty positions of the array with -1s. With this preprocessing, the input dataset **X** will consist of 59,578 vectors of size 12 each, resulting in a 59,578x12 matrix. The values to estimate, **y**, are the labels of each event: proton or muon.

In [ ]:
# Initialize the input and label arrays
X = np.full((len(df), 12), -1, dtype=np.float32)  # 2D array of size (n_event, 12) filled with -1s
y = np.zeros(len(df), dtype=np.float32)  # 1D array of size (n_event,)

# Fill in the input and label arrays
for event_n, event in df.iterrows():
    # Extract the dE/dx values
    NodeOrder = event['NodeOrder']
    Nodededx = event['Nodededx'][NodeOrder]

    # Get up to the first 3 nodes
    nfirstnodes = min(Nodededx.shape[0], 3)
    X[event_n, :nfirstnodes] = Nodededx[:nfirstnodes]

    # Get up to the last 5 nodes
    if Nodededx.shape[0] > nfirstnodes:
        nlastnodes = min(Nodededx.shape[0] - 3, 5)
        X[event_n, nfirstnodes:nfirstnodes + nlastnodes] = Nodededx[-nlastnodes:]

    # Add the global parameters
    X[event_n, -4] = event['TrkLen']
    X[event_n, -3] = event['TrkEDepo']
    X[event_n, -2] = event['TrkDir1']
    X[event_n, -1] = event['TrkDir2']

    # Extract the PID label and add it to the label array
    pid_label = event['TruePID']
    if pid_label == 2212:
        y[event_n] = 0  # protons
    elif pid_label == 13:
        y[event_n] = 1  # muons
    else:
        y[event_n] = 2  # pions

# Standardize the input dataset (mean=0, std=1)
X_stan = scale(X)

To gain insights into the training data, it's a good practice to visualize it first. One way of doing this is by creating a histogram plot for each of the 12 features in our dataset:

In [ ]:
param_names = ['dE/dx node 1', 'dE/dx node 2', 'dE/dx node 3', 'dE/dx node n-4',\
               'dE/dx node n-3', 'dE/dx node n-2', 'dE/dx node n-1', 'dE/dx node n', 'TrkLen',\
               'TrkEDepo', 'TrkDir1', 'TrkDir2']
class_names = ["proton", "muon", "pion"]

# Plot histogram for each feature
plot_parameters(X, y, param_names, class_names, mode="classification")

To evaluate the performance of our neural network model, we need to split the preprocessed dataset into two parts again: one for training and one for testing. We will use the `train_test_split()` function from the sklearn library to do this. 

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_stan[y!=2], y[y!=2], test_size=0.4, 
                                                    random_state=7, shuffle=True)

To accommodate the new dataset, we need to modify the input layer of our network. Once that's done, we can define and train a new model on the modified dataset. Finally, we can evaluate the performance of the model on the test set:

In [ ]:
# Set a random seed for reproducibility
tf.random.set_seed(7)

num_features = 12 # TrkLen, TrkEDepo
num_classes = 1 # One output unit is enough since it's a binary classification problem

# Fully connected neural network model
input_layer = Input(shape=(num_features,)) # Input layer
hidden_layer_1 = Dense(10, activation='relu')(input_layer) # Hidden layer 1
hidden_layer_2 = Dense(10, activation='relu')(hidden_layer_1) # Hidden layer 2
output_layer = Dense(num_classes, activation='sigmoid')(hidden_layer_2) # Output layer
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1)

# Test the model
y_pred = model.predict(X_test).round()
print("Overall accuracy: {:.3f}\n".format(accuracy_score(y_test, y_pred)))
print(" - Proton accuracy: {:.3f}".format(accuracy_score(y_test[y_test==0], y_pred[y_test==0])))
print(" - Muon accuracy: {:.3f}\n".format(accuracy_score(y_test[y_test==1], y_pred[y_test==1])))
conf_matrix = confusion_matrix(y_pred, y_test)
print_conf(conf_matrix, ['protons', 'muons'])

The results are impressive for a binary classification problem. However, it's important to note that our dataset has a third type of particle (pions) that we have ignored, which could be a significant drawback for some applications. Nevertheless, our neural network architecture can be extended to handle multi-class classification problems (where k>2) by modifying the output layer (number of neurons and activation function) and loss function appropriately (**categorical cross-entropy**, which is a generalisation of the binary cross-entropy).

In [ ]:
# Set a random seed for reproducibility
tf.random.set_seed(7)

num_features = 12 # TrkLen, TrkEDepo
num_classes = 3 # Three outputs are needed (multi-class classification problem): proton, muon, and pion

# Fully connected neural network model
input_layer = Input(shape=(num_features,)) # Input layer
hidden_layer_1 = Dense(10, activation='relu')(input_layer) # Hidden layer 1
hidden_layer_2 = Dense(10, activation='relu')(hidden_layer_1) # Hidden layer 2
output_layer = Dense(num_classes, activation='sigmoid')(hidden_layer_2) # Output layer
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.4,
                                                    random_state=7)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1)

# Test the model
y_pred = model.predict(X_test).argmax(axis=1)
print("Overall accuracy: {:2.3}\n".format(accuracy_score(y_test, y_pred)))
print(" - Proton accuracy: {:2.3}".format(accuracy_score(y_test[y_test==0], y_pred[y_test==0])))
print(" - Muon accuracy: {:2.3}".format(accuracy_score(y_test[y_test==1], y_pred[y_test==1])))
print(" - Pion accuracy: {:2.3}\n".format(accuracy_score(y_test[y_test==2], y_pred[y_test==2])))
conf_matrix = confusion_matrix(y_pred, y_test)
print_conf(conf_matrix, ['protons', 'muons', 'pions'])

To increase the capacity of our model and make it more capable of learning complex patterns, we can add more layers and neurons per layer. However, we need to be careful not to overfit the training data and generalize well to unseen data. Regularization techniques, such as dropout and weight decay, can help prevent overfitting. Moreover, it's essential to monitor the performance of the model on a validation set during training to ensure that we are not overfitting:

In [ ]:
# Set seed for reproducibility
tf.random.set_seed(7)

# Define the number of features and classes
num_features = 12 # TrkLen, TrkEDepo
num_classes = 3 # proton, muon, and pion

# Fully connected neural network model
input_layer = Input(shape=(num_features,)) # Input layer
hidden_layer_1 = Dense(100, activation='relu')(input_layer) # Hidden layer 1
hidden_layer_2 = Dense(100, activation='relu')(hidden_layer_1) # Hidden layer 2
hidden_layer_3 = Dense(100, activation='relu')(hidden_layer_2) # Hidden layer 3
hidden_layer_4 = Dense(100, activation='relu')(hidden_layer_3) # Hidden layer 4
output_layer = Dense(num_classes, activation='softmax')(hidden_layer_4) # Output layer
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test loss: {:.3f}".format(loss))
print("Test accuracy: {:.3f}".format(accuracy))

# Make predictions on test set
y_pred = model.predict(X_test).argmax(axis=1)

# Print accuracy for each class and confusion matrix
print("Accuracy for each class:")
print("Proton accuracy: {:.3f}".format(accuracy_score(y_test[y_test==0], y_pred[y_test==0])))
print("Muon accuracy: {:.3f}".format(accuracy_score(y_test[y_test==1], y_pred[y_test==1])))
print("Pion accuracy: {:.3f}".format(accuracy_score(y_test[y_test==2], y_pred[y_test==2])))
conf_matrix = confusion_matrix(y_pred, y_test)
print("Confusion matrix:")
print_conf(conf_matrix, ['protons', 'muons', 'pions'])

## Convolutional neural networks

[Convolutional neural networks (CNNs)](https://direct.mit.edu/neco/article-abstract/1/4/541/5515/Backpropagation-Applied-to-Handwritten-Zip-Code?redirectedFrom=fulltext) have been highly successful in a variety of [HEP tasks](https://iml-wg.github.io/HEPML-LivingReview/), particularly in image classification problems. A CNN is a type of neural network that applies a series of filters, using convolutions, followed by spatial pooling, in sequence to extract increasingly powerful and abstract features from the input image [[citation](http://dl.acm.org/citation.cfm?id=2999134.2999257)].

Each filter in a CNN consists of a set of values that are learned by the network during the training process. CNNs are typically deep neural networks that consist of many convolutional layers, with the output from one convolutional layer forming the input to the next. The last layers of a CNN are usually fully connected layers, where the output layer is followed by a sigmoid or softmax activation function.

To apply CNNs to our dataset, we can convert our 3D events into 2D images. One simple approach is to save the YZ projection of each event as a 2D image. We chose this projection to retain the Z-axis, which corresponds to the beam direction:

In [ ]:
# Set up empty arrays to hold the data
X = np.zeros(shape=(len(df), 58, 189, 1), dtype=np.float32)
y = np.zeros(shape=(len(df),), dtype=np.float32)

# Iterate over the events in the dataset and fill the arrays
for event_n, event in df.iterrows():
    # Get the position and energy deposition information for the event
    NodePosY = event['NodePosY']
    NodePosZ = event['NodePosZ']
    Nodededx = event['Nodededx']

    old_y, old_z, dedxs = -1, -1, []
    # Loop over each voxel in the event and calculate the average energy deposition
    # in each voxel
    for index in range(len(NodePosY)):
        y_coord, z_coord = NodePosY[index], NodePosZ[index]
        y_coord, z_coord = map_value(y_coord, z_coord) # Convert coords into voxels

        # If we are still in the same voxel, add the current energy deposition to the list
        if index == 0 or (y_coord == old_y and z_coord == old_z):
            dedxs.append(Nodededx[index])
        # If we have moved to a new voxel, calculate the average energy deposition in the
        # previous voxel and add it to the X array
        else:
            X[event_n, old_y, old_z, 0] = np.mean(dedxs)
            old_y, old_z, dedxs = y_coord, z_coord, [Nodededx[index]]

    # Calculate the average energy deposition in the last voxel and add it to the X array
    X[event_n, old_y, old_z, 0] = np.mean(dedxs)

    # Set the label for the event
    pid_label = event['TruePID']
    if pid_label == 2212:
        pid_label = 0 # protons
    elif pid_label == 13: 
        pid_label = 1 # muons
    else:
        pid_label = 2 # pions
    y[event_n] = pid_label

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                    random_state=7, shuffle=True)

To visualize the results of the code, we can plot two different views of the same 3D event, along with the corresponding YZ projection. This helps us ensure that the generated 2D images capture the important features of the original 3D events:

In [ ]:
# Set the event number to plot
event_number = 0

# Call the plot_projection function to visualise the event:
plot_projection(df, event_number, X)

We will implement a convolutional neural network (CNN) that consists of several layers to classify the particle type of each event based on its YZ projection. The network will take a YZ projection of an event as input, which will be fed into several convolutional layers followed by max pooling. After that, the output will be flattened and fed into a few fully connected layers. Finally, the output layer will use the softmax activation function to output the predicted probability distribution over the three particle types: protons, muons, and pions:

<div>
<img src="https://cernbox.cern.ch/remote.php/dav/public-files/lJYfLWXBsUaYWnO/cnn.png?scalingup=0&preview=1&a=1&c=284811005316300800%3A0fa0ba41&x=1920&y=1920" width="900"/>
</div>

In [ ]:
# Set seed for reproducibility
tf.random.set_seed(7)

# Define the input shape of the CNN (input image shape)
inp_shape = (58, 189, 1)

# create the input layer
input_layer = Input(shape=inp_shape)

# add the first convolutional layer with 16 filters, filter size of (6, 18), and stride of (2, 3), followed by ReLU activation
x = Conv2D(16, (6,18), padding='valid', strides=(2,3), activation='relu')(input_layer)

# add the first max pooling layer with pool size of (2, 3) and stride of (2, 3)
x = MaxPooling2D(pool_size=(2,3), strides=(2,3))(x)

# add the second convolutional layer with 32 filters, filter size of (3, 3), and stride of (2, 3), followed by ReLU activation
x = Conv2D(32, (3,3), padding='valid', strides=(2,3), activation='relu')(x)

# add the second max pooling layer with pool size of (2, 3) and stride of (2, 3)
x = MaxPooling2D(pool_size=(2,3), strides=(2,3))(x)

# flatten the 3D output to 1D
x = Flatten()(x)

# add a fully connected layer with 64 units and ReLU activation
x = Dense(64, activation='relu')(x)

# add the output layer with 3 units and softmax activation for multiclass classification
output = Dense(3, activation='softmax')(x)

# create the model with the input and output layers
model = Model(inputs=input_layer, outputs=output)

# compile the model with sparse categorical cross-entropy loss, Adam optimizer, and accuracy as the metric
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# print the model summary
model.summary()

It's great to see the CNN coming together! Now let's train the model and see how it performs:

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=1)

# Test the model
y_pred = model.predict(X_test).argmax(axis=1)
print("Overall accuracy: {:2.3}\n".format(accuracy_score(y_test, y_pred)))
print(" - Proton accuracy: {:2.3}".format(accuracy_score(y_test[y_test==0], y_pred[y_test==0])))
print(" - Muon accuracy: {:2.3}".format(accuracy_score(y_test[y_test==1], y_pred[y_test==1])))
print(" - Pion accuracy: {:2.3}\n".format(accuracy_score(y_test[y_test==2], y_pred[y_test==2])))
conf_matrix = confusion_matrix(y_pred, y_test)
print_conf(conf_matrix, ['protons', 'muons', 'pions'])

It's important to note that the CNN was trained on a much more limited piece of information compared to the fully-connected network (FCN). While the FCN was trained on many well understood physics informations, such as the track length, direction, and energy deposited, the CNN was only trained on 2D signatures of the tracks. However, this doesn't necessarily mean that the CNN is worse than the FCN. It's important for scientists to understand which method is best for each situation, and in this case, the CNN was able to achieve a respectable accuracy given its limited input information. In fact, by adding capacity to the CNN by designing wider and deeper networks, it's possible to improve upon the results achieved by the FCN.

##Extra work!

To improve the performance of the models, one can increase their capacity by designing wider networks (adding more neurons or convolutional filters per layer) and deeper networks (adding more layers). Experimenting with different architectures and hyperparameters is crucial for achieving better results.

Useful links:

- How to Control Neural Network Model Capacity With Nodes and Layers: https://machinelearningmastery.com/how-to-control-neural-network-model-capacity-with-nodes-and-layers/.
- TensorFlow 2 quickstart for beginners: https://www.tensorflow.org/tutorials/quickstart/beginner.
- Building a Convolutional Neural Network Using TensorFlow – Keras: https://www.analyticsvidhya.com/blog/2021/06/building-a-convolutional-neural-network-using-tensorflow-keras/.